In [2]:
from openai import OpenAI
client = OpenAI()

In [9]:
# Upload a file with an "assistants" purpose
# files = ['contacts.csv', 'searches.csv']
# files = ["column_descriptions.txt"]
uploaded_files = [] 

# for f in files:
#   file = client.files.create(
#     file=open(f"../data/{f}", "rb"),
#     purpose='assistants'
#   )
#   uploaded_files.append(file)

file_ids = ['file-4FKpqlvJOZqxNUfIJHAHRGJY', 'file-TzOVAHko5TwyrRsaPySmIR0Z', 'file-LbOzhUfzfTVqYNRUOPderfwy'] # contacts, searches, column_descriptions
file_ids



['file-4FKpqlvJOZqxNUfIJHAHRGJY',
 'file-TzOVAHko5TwyrRsaPySmIR0Z',
 'file-LbOzhUfzfTVqYNRUOPderfwy']

In [20]:
# Create an assistant using the file ID
# assistant = client.beta.assistants.create(
#   instructions="You are a personal assistant helping users visualize and analyze trends in data. You have access to a dataset of user searches and contacts. You also have a text file containing information regarding the column names and their meanings. You should help users understand the data and answer questions about it.",
#   model="gpt-4-turbo",
#   tools=[{"type": "code_interpreter"}],
#   file_ids=file_ids
# )

In [18]:
message

Message(id='msg_Bg5OFYk0ractCLRhYpnXa3NV', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='which day of the week were the most bookings made? Display a seaborn plot to help me visualize this.'), type='text')], created_at=1713078583, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_1aJB28qZU9kwF0o9J5Xgn7tg')

In [5]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          print(output.type)
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
          elif output.type == "image":
            print(type(output.image))
            image_id = output.image.file_id
            print(image_id)
            image_data = client.files.content(image_id)
            image_data_bytes = image_data.read()

            with open("./my-image.png", "wb") as file:
                file.write(image_data_bytes)
 
# Then, we use the `create_and_stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 


In [9]:
thread = client.beta.threads.create()


message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What day of the week were the most bookings made? Display a seaborn plot to help me visualize this."
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id="asst_gwH0YPXzLZpaUoNnGQzmkOaG",
  instructions="Answer with minimal text. Also, output image data, if it exists",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import pandas as pd

# Load the data from the uploaded files
df1 = pd.read_csv('/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY')
df2 = pd.read_csv('/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z')
df3 = pd.read_csv('/mnt/data/file-LbOzhUfzfTVqYNRUOPderfwy')

# Show first few rows of each dataframe to understand their structure
df1.head(), df2.head(), df3.head()

output >
logs

---------------------------------------------------------------------------
ParserError                               Traceback (most recent call last)
Cell In[1], line 6
      4 df1 = pd.read_csv('/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY')
      5 df2 = pd.read_csv('/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z')
----> 6 df3 = pd.read_csv('/mnt/data/file-LbOzhUfzfTVqYNRUOPderfwy')
      8 # Show first few rows of each dataframe to understand their structure
      9 df1.head(), df2.head(), df3.head()

File ~/.local/lib/python3.11/site-packages/pandas/util/_decorators.py:211, in deprecate_kwarg.<locals>.